In [3]:
# Data Loading and Preprocessing
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, Input, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import BatchNormalization

print("--- Data Loading and Preprocessing ---")

def one_hot_encode_sequence(sequence, max_len):
    mapping = {
        'A': [1, 0, 0, 0], 'a': [1, 0, 0, 0],
        'T': [0, 1, 0, 0], 't': [0, 1, 0, 0],
        'G': [0, 0, 1, 0], 'g': [0, 0, 1, 0],
        'C': [0, 0, 0, 1], 'c': [0, 0, 0, 1],
        'N': [0, 0, 0, 0], 'n': [0, 0, 0, 0]
    }
    encoded = [mapping.get(char, [0, 0, 0, 0]) for char in sequence]
    encoded = np.array(encoded, dtype=np.float32)
    if len(encoded) < max_len:
        pad = np.zeros((max_len - len(encoded), 4), dtype=np.float32)
        encoded = np.vstack((encoded, pad))
    return encoded[:max_len]

train_df = pd.read_csv('dm3.kc167.tads.train.csv', header=None)
X_train_raw, y_train = train_df[0].values, train_df[1].values
test_df = pd.read_csv('dm3.kc167.tads.test.csv', header=None)
X_test_raw, y_test = test_df[0].values, test_df[1].values

sequence_length = len(X_train_raw[0])
X_train = np.array([one_hot_encode_sequence(seq, sequence_length) for seq in X_train_raw])
X_test = np.array([one_hot_encode_sequence(seq, sequence_length) for seq in X_test_raw])
print("Train shape:", X_train.shape, "Test shape:", X_test.shape)
input_shape = (sequence_length, 4)

# --- Callbacks (defined globally as they manage the training process, not model internal parameters) ---
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001, verbose=1)

print("\n--- Training Models ---")

--- Data Loading and Preprocessing ---
Train shape: (28140, 1000, 4) Test shape: (2000, 1000, 4)

--- Training Models ---


In [8]:
# Model 1 	Model 1 – Simple CNN
# Adjusted hyperparameters for Model 1 to improve test accuracy while managing overfitting
print("\n--- Model 1: Simple CNN ---")
# Model-specific hyperparameters
model1_learning_rate = 0.001
model1_epochs = 100
model1_batch_size = 64
model1_validation_split = 0.2
model1_dropout_rate = 0.3 # Keeping dropout at 0.3
model1_l2_strength = 0.0005 # Reverting L2 strength to 0.0005

model1 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=64, kernel_size=9, activation='relu', kernel_regularizer=l2(model1_l2_strength)), # Increased filters from 32 to 64
    MaxPooling1D(pool_size=2),
    Dropout(model1_dropout_rate),
    Flatten(),
    Dense(units=64, activation='relu', kernel_regularizer=l2(model1_l2_strength)), # Increased Dense units from 32 to 64
    Dropout(model1_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model1.compile(optimizer=Adam(model1_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=model1_epochs, batch_size=model1_batch_size, validation_split=model1_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model1.evaluate(X_test, y_test, verbose=0)
print(f"Model 1 Test Accuracy: {accuracy:.4f}")


--- Model 1: Simple CNN ---
Epoch 1/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.5514 - loss: 0.8122 - val_accuracy: 0.0027 - val_loss: 1.0319 - learning_rate: 0.0010
Epoch 2/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.6460 - loss: 0.7051 - val_accuracy: 0.3564 - val_loss: 0.9217 - learning_rate: 0.0010
Epoch 3/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.7193 - loss: 0.6890 - val_accuracy: 0.5593 - val_loss: 0.8884 - learning_rate: 0.0010
Epoch 4/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.7693 - loss: 0.6695 - val_accuracy: 0.1501 - val_loss: 1.6729 - learning_rate: 0.0010
Epoch 5/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8094 - loss: 0.6488 - val_accuracy: 0.2745 - val_loss: 1.5681 - learning_rate: 0.0010
Epoch 6/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8387 - loss: 0.6381 - val_accuracy: 0.3826 - val_loss: 1.5536 - learning_rate: 0.0010
Epoch 7/100
352/352 ━━━━━━━━━━━━━━━━━━━━ 8s 23m

In [ ]:
# Model 2 Model 2 – Deeper CNN
print("\n--- Model 2: Deeper CNN ---")
# Model-specific hyperparameters
model2_learning_rate = 0.001
model2_epochs = 100
model2_batch_size = 64
model2_validation_split = 0.2
model2_dropout_rate = 0.3
model2_l2_strength = 0.0005

model2 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=16, kernel_size=9, activation='relu', kernel_regularizer=l2(model2_l2_strength)),
    MaxPooling1D(pool_size=2), Dropout(model2_dropout_rate),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(model2_l2_strength)),
    MaxPooling1D(pool_size=2), Dropout(model2_dropout_rate),
    Flatten(),
    Dense(units=64, activation='relu', kernel_regularizer=l2(model2_l2_strength)),
    Dropout(model2_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model2.compile(optimizer=Adam(model2_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, epochs=model2_epochs, batch_size=model2_batch_size, validation_split=model2_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model2.evaluate(X_test, y_test, verbose=0)
print(f"Model 2 Test Accuracy: {accuracy:.4f}")

In [ ]:
# Model 3 	Model 3 – Simple LSTM
print("\n--- Model 3: Simple LSTM ---")
# Model-specific hyperparameters
model3_learning_rate = 0.001
model3_epochs = 100
model3_batch_size = 64
model3_validation_split = 0.2
model3_dropout_rate = 0.3
model3_l2_strength = 0.0005

model3 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10)),
    Dropout(model3_dropout_rate),
    Dense(units=32, activation='relu', kernel_regularizer=l2(model3_l2_strength)),
    Dropout(model3_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model3.compile(optimizer=Adam(model3_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, epochs=model3_epochs, batch_size=model3_batch_size, validation_split=model3_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model3.evaluate(X_test, y_test, verbose=0)
print(f"Model 3 Test Accuracy: {accuracy:.4f}")

In [ ]:
# Model 4 Model 4 – Deeper LSTM
print("\n--- Model 4: Deeper LSTM ---")
# Model-specific hyperparameters
model4_learning_rate = 0.001
model4_epochs = 100
model4_batch_size = 64
model4_validation_split = 0.2
model4_dropout_rate = 0.3
model4_l2_strength = 0.0005

model4 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10, return_sequences=True)),
    Dropout(model4_dropout_rate),
    Bidirectional(LSTM(units=10)),
    Dropout(model4_dropout_rate),
    Dense(units=64, activation='relu', kernel_regularizer=l2(model4_l2_strength)),
    Dropout(model4_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model4.compile(optimizer=Adam(model4_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model4.fit(X_train, y_train, epochs=model4_epochs, batch_size=model4_batch_size, validation_split=model4_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model4.evaluate(X_test, y_test, verbose=0)
print(f"Model 4 Test Accuracy: {accuracy:.4f}")

In [ ]:
# Model 5 CNN-LSTM Hybrid
print("\n--- Model 5: CNN-LSTM Hybrid ---")
# Model-specific hyperparameters
model5_learning_rate = 0.001
model5_epochs = 100
model5_batch_size = 64
model5_validation_split = 0.2
model5_dropout_rate = 0.3
model5_l2_strength = 0.0005

model5 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(model5_l2_strength)),
    MaxPooling1D(pool_size=2), Dropout(model5_dropout_rate),
    Bidirectional(LSTM(units=10)),
    Dropout(model5_dropout_rate),
    Dense(units=32, activation='relu', kernel_regularizer=l2(model5_l2_strength)),
    Dropout(model5_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model5.compile(optimizer=Adam(model5_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model5.fit(X_train, y_train, epochs=model5_epochs, batch_size=model5_batch_size, validation_split=model5_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model5.evaluate(X_test, y_test, verbose=0)
print(f"Model 5 Test Accuracy: {accuracy:.4f}")

In [ ]:
# Model 6 CNN with More Dense Layers
print("\n--- Model 6: CNN with More Dense Layers ---")
# Model-specific hyperparameters
model6_learning_rate = 0.001
model6_epochs = 100
model6_batch_size = 64
model6_validation_split = 0.2
model6_dropout_rate = 0.3
model6_l2_strength = 0.0005

model6 = Sequential([
    Input(shape=input_shape),
    Conv1D(filters=32, kernel_size=9, activation='relu', kernel_regularizer=l2(model6_l2_strength)),
    MaxPooling1D(pool_size=2), Dropout(model6_dropout_rate), Flatten(),
    Dense(units=64, activation='relu', kernel_regularizer=l2(model6_l2_strength)),
    Dropout(model6_dropout_rate),
    Dense(units=32, activation='relu', kernel_regularizer=l2(model6_l2_strength)),
    Dropout(model6_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model6.compile(optimizer=Adam(model6_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model6.fit(X_train, y_train, epochs=model6_epochs, batch_size=model6_batch_size, validation_split=model6_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model6.evaluate(X_test, y_test, verbose=0)
print(f"Model 6 Test Accuracy: {accuracy:.4f}")

In [ ]:
# Model 7 LSTM with More Dense Layers
print("\n--- Model 7: LSTM with More Dense Layers ---")
# Model-specific hyperparameters
model7_learning_rate = 0.001
model7_epochs = 100
model7_batch_size = 64
model7_validation_split = 0.2
model7_dropout_rate = 0.3
model7_l2_strength = 0.0005

model7 = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(units=10)),
    Dropout(model7_dropout_rate),
    Dense(units=64, activation='relu', kernel_regularizer=l2(model7_l2_strength)),
    Dropout(model7_dropout_rate),
    Dense(units=32, activation='relu', kernel_regularizer=l2(model7_l2_strength)),
    Dropout(model7_dropout_rate),
    BatchNormalization(),
    Dense(units=1, activation='sigmoid')
])
model7.compile(optimizer=Adam(model7_learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model7.fit(X_train, y_train, epochs=model7_epochs, batch_size=model7_batch_size, validation_split=model7_validation_split,
           callbacks=[early_stopping, reduce_lr], verbose=1)
loss, accuracy = model7.evaluate(X_test, y_test, verbose=0)
print(f"Model 7 Test Accuracy: {accuracy:.4f}")